In [96]:
import pandas as pd
from urllib.parse import unquote

dataFolder = '../Data/wikispeedia_paths-and-graph/'
articlesDf = pd.read_csv(dataFolder+ 'articles.tsv')
articlesDf = articlesDf.iloc[10:]
articlesSeries  = pd.Series(articlesDf.index.map(lambda x : x[0])).map(lambda y : unquote(y,'utf-8'))
articlesSeries.columns= 'article'
articlesSeries.head(10)

linksDf = pd.read_csv(dataFolder + 'links.tsv', skiprows= 11, sep = '\t')
linksDf.columns = ['linkSource', 'linkTarget']
linksDf['linkSource'] = linksDf['linkTarget'].map(lambda x : unquote(x, 'utf-8'))
linksDf['linkTarget'] = linksDf['linkTarget'].map(lambda x : unquote(x, 'utf-8'))
linksDf

categoriesDf = pd.read_csv(dataFolder + 'categories.tsv', skiprows=12, sep= '\t')
categoriesDf.columns = ['article', 'categories']
categoriesDf['article'] = categoriesDf['article'].map(lambda x : unquote(x,'utf-8'))


#for paths_finishedDf FORMAT:   hashedIpAddress   timestamp   durationInSec   path   rating
paths_finishedDf = pd.read_csv(dataFolder + 'paths_finished.tsv',skiprows= 15, sep = '\t')
paths_finishedDf.columns = ['hashIpAddress', 'timestamp', 'durationInSec', 'path', 'rating']

paths_unfinishedDf = pd.read_csv(dataFolder + 'paths_unfinished.tsv',skiprows = 16, sep = '\t')
paths_unfinishedDf.columns = ['hashIpAddress', 'timestamp', 'durationInSec', 'path', 'target','type']
categoriesDf

shortestPath = pd.read_csv(dataFolder+'shortest-path-distance-matrix.txt', skiprows= 15)
print(shortestPath.shape)
shortestPath.columns = ['shortest_path', 'temp']
shortestPath = shortestPath['shortest_path']
shortestPathDf = pd.DataFrame(columns= articlesSeries)
shortestPathDf


(4604, 2)


,Áedán_mac_Gabráin,Åland,Édouard_Manet,Éire,Óengus_I_of_the_Picts,€2_commemorative_coins,10th_century,11th_century,12th_century,13th_century,...,Ziad_Jarrah,Zimbabwe,Zinc,Zinc_chloride,Zion_National_Park,Zionism,Zirconium,Zoroaster,Zuid-Gelders,Zulu
